In [41]:
import os
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [49]:
HAM_DIR = './Dataset/ham/'
SPAM_DIR = './Dataset/spam/'
HAM_CLEAN_DIR = './Dataset/ham_clean/'
SPAM_CLEAN_DIR = './Dataset/spam_clean/'
FEATURES_FILE_DIR = './Dataset/features.txt'
ENG_VOCAB_FEATURES_FILE_DIR = './Dataset/eng_vocab_features.txt'
CLEANR_HTML = re.compile('(<.*?>|&[a-zA-Z]+;)')
URL = re.compile("((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*")
EMAIL = re.compile('[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}')
NUMBER = re.compile('[0-9]+')
DOLLAR = re.compile("\$")
SPACE = re.compile("( |\t)+")
english_vocab = set(w.lower() for w in nltk.corpus.words.words())
lemmatizer = WordNetLemmatizer()
eng_stop_words = stopwords.words('english')

In [3]:
for _ ,__ ,files in os.walk(HAM_DIR):
    print("files len : ",len(files))


files len :  6951


In [4]:
def walk_till_empty(lines , i):
    while i+1 < len(lines) and lines[i]!="\n":
       i+=1
    return i   

In [5]:
def extract_email(filename):
    with open(filename,encoding = "ISO-8859-1") as f:
           lines = f.readlines()
        
    i = walk_till_empty(lines, 0)
    while len(lines[i+1].strip().split(" ")[0].lower()) == 0 or (lines[i+1].strip().split(" ")[0].lower()[0] >= 'a' and lines[i+1].strip().split(" ")[0].lower()[0] <= 'z' and (not re.match(URL,lines[i+1]) and lines[i+1].strip().split(" ")[0].find(":") != -1)):
        i = walk_till_empty(lines, i+1)

    return lines[i+1:]


In [ ]:
[line for line in extract_email('./Dataset/spam/00005.ed0aba4d386c5e62bc737cf3f0ed9589') if(not len(line.strip()) == 0)]

In [6]:
words = {}

In [17]:
def create_ham_clean():
    for _ ,__ ,ham_files in os.walk(HAM_DIR):
        print("ham files found : ",len(files))
    n = 0
    for ham_file in ham_files :
        outF = open(f"{HAM_CLEAN_DIR}ham{n}.txt", "w",encoding = "ISO-8859-1")
        n+=1
        glob = ""
        for line in extract_email(f"{HAM_DIR}{ham_file}"):
            if(not len(line.strip()) == 0):
                glob+=" "+line
        
        line = re.sub(SPACE," ",glob.lower().replace("\n"," ").strip())
        
        line = re.sub(CLEANR_HTML, '', line)
        line = re.sub(URL,"httpaddr",line)
        line = re.sub(EMAIL,"emailaddr",line)
        line = re.sub(NUMBER,"number",line)
        line = re.sub(DOLLAR,"dollar",line)
        
        # NLTK JOB
        line = [lemmatizer.lemmatize(word) for word in word_tokenize(line) if word not in eng_stop_words]
        
        line = " ".join(line)
        line = line.translate(str.maketrans('', '', string.punctuation))
        tokenized = word_tokenize(line)
        for word in tokenized :
            if word not in words.keys() :
                words[word] = 1
            else :
                words[word]+= 1

        print(line,file=outF)


            
        outF.close()

    

In [22]:
def create_spam_clean():
    for _ ,__ ,spam_files in os.walk(SPAM_DIR):
        print("spam files found : ",len(spam_files))
    n = 0
    for spam_file in spam_files :
        outF = open(f"{SPAM_CLEAN_DIR}spam{n}.txt", "w",encoding = "ISO-8859-1")
        n+=1
        glob = ""
        try :
            mail = extract_email(f"{SPAM_DIR}{spam_file}")
            for line in mail:
                if(not len(line.strip()) == 0):
                   glob+=" "+line   
        except Exception:
            print(spam_file)
        
        
        line = re.sub(SPACE," ",glob.lower().replace("\n"," ").strip())
        
        line = re.sub(CLEANR_HTML, '', line)
        line = re.sub(URL,"httpaddr",line)
        line = re.sub(EMAIL,"emailaddr",line)
        line = re.sub(NUMBER,"number",line)
        line = re.sub(DOLLAR,"dollar",line)
        
        # NLTK JOB
        line = [lemmatizer.lemmatize(word) for word in word_tokenize(line) if word not in eng_stop_words]
        line = " ".join(line)
        line = line.translate(str.maketrans('', '', string.punctuation))
        tokenized = word_tokenize(line)
        for word in tokenized :
            if word not in words.keys() :
                words[word] = 1
            else :
                words[word]+= 1

        print(line,file=outF)


            
        outF.close()

    

In [19]:
create_ham_clean()

ham files found :  6951


In [23]:
create_spam_clean()

spam files found :  2397


In [21]:
len(words)

76928

In [29]:
sortedwords = {k: v for k, v in sorted(words.items(), key=lambda item: item[1],reverse=True) if v > 100}

In [ ]:
sortedwords.items()


In [36]:
features = ' '.join([key for key in sortedwords.keys()])

In [39]:
featuresFile = open(FEATURES_FILE_DIR, "w",encoding = "ISO-8859-1")
print(features,file=featuresFile)
featuresFile.close()

In [53]:
sorted_english_vocab_features = {k: v for k, v in sorted(words.items(), key=lambda item: item[1],reverse=True) if v > 100 and k in english_vocab and len(k)>=2}

In [54]:
len(sorted_english_vocab_features)

2341

In [55]:
eng_vocab_features = ' '.join([key for key in sorted_english_vocab_features.keys()])

In [56]:
eng_vocab_featuresFile = open(ENG_VOCAB_FEATURES_FILE_DIR, "w",encoding = "ISO-8859-1")
print(eng_vocab_features,file=eng_vocab_featuresFile)
eng_vocab_featuresFile.close()